# 🧩 1.3 – Cuantificadores y Grupos

En este notebook aprenderás a controlar **cuántas veces se repite un patrón** y cómo **agrupar partes específicas** del texto mediante **grupos de captura**.

---
## 🎯 Objetivos
- Comprender el uso de `*`, `+`, `?`, `{n}`, `{n,m}`.
- Entender la diferencia entre grupos capturantes `( )` y no capturantes `(?: )`.
- Acceder a los grupos mediante el objeto `Match`.
- Aplicar estos conceptos en la extracción de datos estructurados (ej. correos electrónicos).

> 💡 **Recuerda:** ejecuta las celdas en orden para mantener las variables disponibles.

---
## 1️⃣ Cuantificadores básicos

| Símbolo | Significado | Ejemplo |
|:--------:|:------------|:--------|
| `*` | Cero o más repeticiones | `ba*` → b, ba, baa, baaa |
| `+` | Una o más repeticiones | `ba+` → ba, baa |
| `?` | Cero o una repetición | `colou?r` → color o colour |
| `{n}` | Exactamente n repeticiones | `\d{4}` → 4 dígitos |
| `{n,m}` | Entre n y m repeticiones | `a{2,4}` → aa, aaa, aaaa |

In [1]:
import re

texto = "ba baa baaa baaaa"

print(re.findall(r"ba*", texto))   # cero o más 'a'
print(re.findall(r"ba+", texto))   # una o más 'a'
print(re.findall(r"ba{2,3}", texto)) # entre 2 y 3 'a'

['ba', 'baa', 'baaa', 'baaaa']
['ba', 'baa', 'baaa', 'baaaa']
['baa', 'baaa', 'baaa']


---
## 2️⃣ Grupos de captura `( )`

Los paréntesis permiten **agrupar y capturar** partes específicas del texto.

Por ejemplo, para capturar el número y las letras de una matrícula española (`1234-ABC`):

In [2]:
texto = "Coches: 1234-ABC, 5555-XYZ"
patron = r"(\d{4})-([A-Z]{3})"

coincidencias = re.findall(patron, texto)
print(coincidencias)  # [('1234', 'ABC'), ('5555', 'XYZ')]

[('1234', 'ABC'), ('5555', 'XYZ')]


✅ Cada coincidencia devuelve una **tupla** con el contenido de los grupos.

Si solo quieres agrupar sin capturar, usa **grupos no capturantes**: `(?: ... )`.

In [3]:
texto = "aaa bbb ccc"
print(re.findall(r"(a+)", texto))     # grupo capturante
print(re.findall(r"(?:a+)", texto))   # grupo no capturante

['aaa']
['aaa']


---
## 3️⃣ El objeto `Match`

Cuando usamos `re.search()` o `re.match()`, obtenemos un objeto `Match` que guarda los grupos encontrados.

Podemos acceder a ellos mediante:
- `.group(n)` → texto del grupo *n*.
- `.groups()` → todos los grupos.
- `.start()` / `.end()` → posiciones en el texto.

In [4]:
m = re.search(r"(\d{4})-([A-Z]{3})", "Matrícula: 4321-DFG")

if m:
    print("Coincidencia completa:", m.group(0))
    print("Grupo 1 (números):", m.group(1))
    print("Grupo 2 (letras):", m.group(2))
    print("Posición en texto:", m.span())

Coincidencia completa: 4321-DFG
Grupo 1 (números): 4321
Grupo 2 (letras): DFG
Posición en texto: (11, 19)


---
## 4️⃣ Ejercicio guiado – Extraer nombre y dominio de correos electrónicos

Tenemos un texto con correos electrónicos y queremos separar:
- **nombre de usuario**
- **dominio** (ej. `gmail.com`)

💡 *Pista:* usa dos grupos `( ... )@( ... )` y el cuantificador `+`.

In [5]:
texto = "Correos: ana@gmail.com, juan.perez@empresa.es, maria-99@uni.edu"

# TODO: escribe tu patrón aquí
# patron = r"( ... )@( ... )"
# resultados = re.findall(patron, texto)
# print(resultados)

# Resultado esperado:
# [('ana', 'gmail.com'), ('juan.perez', 'empresa.es'), ('maria-99', 'uni.edu')]

---
## 5️⃣ Solución sugerida
Puedes comprobar tu resultado con esta celda:

In [6]:
patron = r"([\w\.-]+)@([\w\.-]+)"
resultados = re.findall(patron, texto)
print(resultados)

[('ana', 'gmail.com'), ('juan.perez', 'empresa.es'), ('maria-99', 'uni.edu')]


---
## 6️⃣ Bonus – Validación con grupos

Podemos usar grupos también en validaciones, por ejemplo para comprobar si un email es válido:

`^[\w\.-]+@[\w\.-]+\.\w+$`

In [7]:
correos = ["usuario@mail.com", "nombre@empresa", "maria99@uni.edu"]
patron = r"^[\w\.-]+@[\w\.-]+\.\w+$"

for c in correos:
    print(c, "✅" if re.match(patron, c) else "❌")

usuario@mail.com ✅
nombre@empresa ❌
maria99@uni.edu ✅


---
## 7️⃣ Resumen del notebook

- Los **cuantificadores** controlan el número de repeticiones.
- Los **grupos `( )`** permiten extraer partes concretas del texto.
- Los **grupos no capturantes `(?: )`** agrupan sin devolver resultados.
- El **objeto Match** proporciona métodos útiles (`group()`, `span()`, etc.).

�� Dominar los grupos es esencial para estructurar y validar texto en cualquier contexto (correos, logs, formularios, etc.).

---
**Fin del notebook 1.3 – Cuantificadores y Grupos.**